# Chapter 9 A pythonic object (2)

## Python의 정보 은닉 방식
class의 attribute, method 에 대한 접근을 제어할 수 있는 기능  

1. `private` : 해당 클래스에서만 접근 가능
2. `protected` : 해당 클래스 또는 해당 클래스를 상속 받은 클래스에서만 접근 가능
3. `public` : 어떤 클래스라도 접근 가능, 기본적으로 모든 attribute, method는 public 즉, 클래스 외부에서 접근 가능

In [39]:
from array import array 
import math

class Vector2d: 
    typecode = 'd'

    def __init__(self, x, y): 
        self.__x = float(x) 
        self.__y = float(y)

    @property
    def x(self): 
        return self.__x

    @property
    def y(self): 
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self): 
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''): 
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords) 
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode) 
        return cls(*memv)

### Private

Python에는 다른 언어처럼 접근제어자가 존재하지 않는다. 하지만 분명히 접근에 대한 제한이 필요한 상황이 있고, 이를 실현하기 위한 다른 방법이 존재한다. 가령, 사용자가 클래스 내의 속성에 대하여 모르는 상태에서 어떤 속성을 할당하는 상황을 생각해보자. 만약 그 속성이 이미 클래스 내에 존재하고, 중요한 역할을 하는 경우에 프로그램에 치명적인 영향을 끼칠 수도 있다.  
이러한 경우를 대비하여, 속성을 `__attribute` 와 같이 underbar를 앞에 2개 붙히는 방법으로 이름을 지어주면 된다. (뒤에 underbar를 1개까지 추가할 수 있다. 2개 추가하면 public이 되므로 주의)

In [40]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [42]:
print(v1._Vector2d__x)
v1._Vector2d__x = 5.
print(v1._Vector2d__x)
v1.__x

5.0
5.0


AttributeError: 'Vector2d' object has no attribute '__x'

In [43]:
class Vector2d_inherited(Vector2d):
    def blahblah(self):
        print(self.__x)
v2 = Vector2d_inherited(5, 6)
v2.blahblah()

AttributeError: 'Vector2d_inherited' object has no attribute '_Vector2d_inherited__x'

In [44]:
class Vector2d_inherited(Vector2d):
    def blahblah(self):
        print(self._Vector2d__x)
v2 = Vector2d_inherited(5, 6)
v2.blahblah()

5.0


### Protected
해당 속성의 앞에 underbar를 1개 붙여서 표시하고, 실제로 제약되는 것은 없고 일종의 경고 표시로 사용한다.

In [45]:
class TimeNow:
    def __init__(self, hour, minute, country):
        self._hour = hour
        self._minute = minute
        self._country = country
    
    def get_time(self):
        return "It is %d:%d" % (self._hour, self._minute)
    
    def _set_time(self, hour, minute):
        self._hour = hour
        self._minute = minute

In [46]:
time = TimeNow(12, 30, "Korea")
print(time._hour, time._minute)
print(time.get_time())
time._set_time(1, 23)
print(time.get_time())

12 30
It is 12:30
It is 1:23


## __slots__ 클래스 속성을 사용한 메모리 절약

Python의 모든 클래스는 인스턴스 속성을 가지고, 이를 관리하기 위해 내부적으로 딕셔너리를 사용한다. 객체의 `__dict__` 에 접근하여 인스턴스 속성들을 딕셔너리 형태로 확인할 수 있다.  
일정한 속성들로 구성된 클래스들의 경우, 딕셔너리를 사용하면 메모리를 낭비하게 될 수 있다. 클래스를 정의할 때, `__slots__` 라는 변수를 설정해서 해당 클래스에 의해 만들어진 객체의 인스턴스 속성 관리에 딕셔너리 대신 set을 사용하도록 할 수 있다. 

In [47]:
class TimeNow_with_slots:
    __slots__ = ('_hour', '_minute', '_country')
    def __init__(self, hour, minute, country):
        self._hour = hour
        self._minute = minute
        self._country = country
    
    def get_time(self):
        return "It is %d:%d" % (self._hour, self._minute)
    
    def _set_time(self, hour, minute):
        self._hour = hour
        self._minute = minute

In [48]:
print(time.__dict__)
time_with_slots = TimeNow_with_slots(2, 34, 'Korea')
print(time_with_slots.__slots__)
print(time_with_slots.__dict__)

{'_hour': 1, '_minute': 23, '_country': 'Korea'}
('_hour', '_minute', '_country')


AttributeError: 'TimeNow_with_slots' object has no attribute '__dict__'

### __slots__ 를 사용할 때 주의사항
1. 클래스를 상속할 때마다 `__slots__` 를 다시 정의해주어야 한다. 
2. 인스턴스는 `__slots__` 에 명시 된 속성만을 가질 수 있다. 이를 해결하기 위해서는 `__slots__` 내에 `__dict__` 를 포함하면 되지만, 이렇게 하면 `__slots__` 를 사용하는 목적(메모리 절약)을 잃게 된다. 
3. 인스턴스에 대한 약한 참조가 불가능하게 된다. 해결을 위해서는 `__slots__` 에 `__weakref__` 를 포함해야한다. 

## Overriding Class Attributes

cf) 클래스 속성 vs 인스턴스 속성  
클래스 속성은 클래스에 속해있으며, 모든 인스턴스에서 공유한다. 인스턴스 전체가 사용해야 하는 값을 저장할 때 사용한다. 반면 인스턴스 속성은 인스턴스별로 독립되어있고, 각 인스턴스가 값을 따로 저장해야할 때 사용한다. 

클래스 속성은 인스턴스 속성의 기본 설정 값으로 사용될 수 있다. 만약 사용자가 해당 인스턴스 속성을 새로운 값으로 설정해도 해당 인스턴스 속성이 새로 생길 뿐, 원래의 클래스 속성 값은 변하지 않고 유지된다. 이러한 성질을 이용해 사용자가 인스턴스를 커스터마이징하여 사용할 수 있다. 

In [57]:
v3 = Vector2d(1.1, 2.2)
dumpd = bytes(v3)
print(dumpd)
print(len(dumpd))

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'
17


In [58]:
v3.typecode = 'f'
dumpf = bytes(v3)
print(dumpf)
print(len(dumpf))
print(Vector2d.typecode)

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'
9
d


위처럼 설정하여 사용할 수도 있고, 이 같은 속성 값을 자주 사용하게 될 경우, 아래와 같이 클래스를 상속 받아 클래스 속성을 새로 설정하여 사용하는 것이 더 효율적이다. 

In [59]:
class ShortVector2d(Vector2d):
    typecode = 'f'
sv = ShortVector2d(1/11, 1/27)
print(sv)
print(len(bytes(sv)))

(0.09090909090909091, 0.037037037037037035)
9
